In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from pathlib import Path
from google.colab import drive

drive.mount("/content/drive")

spotify_df = pd.read_csv(Path("/content/drive/My Drive/spotify-2023_updated.csv"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
spotify_df = spotify_df.drop(["track_name", "artist_name", "in_spotify_charts", "in_apple_charts"], axis=1)

In [3]:
df = spotify_df.drop("mode", axis = 1)

dummies_df = pd.get_dummies(spotify_df["mode"])

final_df = pd.concat([df, dummies_df], axis=1)
final_df.head()

,artist_count,released_year,released_month,released_day,streams,bpm,danceability_pct,valence_pct,energy_pct,acousticness_pct,instrumentalness_pct,liveness_pct,speechiness_pct,Major,Minor
0,2,2023,7,14,141381703,125,80,89,83,31,0,8,4,1,0
1,1,2023,3,23,133716286,92,71,61,74,7,0,10,4,1,0
2,1,2023,6,30,140003974,138,51,32,53,17,0,31,6,1,0
3,1,2019,8,23,800840817,170,55,58,72,11,0,11,15,1,0
4,1,2023,5,18,303236322,144,65,23,80,14,63,11,6,0,1


In [4]:
streaming_bin = [0, 250000000, 3803895074]
label = [int("0"), int("1")]

final_df["song_status"] = pd.cut(final_df["streams"], streaming_bin, labels=label)
final_df["song_status"] = final_df["song_status"].astype(int)
final_df.dtypes

final_df.drop("streams", axis = 1, inplace=True)

# 0 = bad song, 1 = good song

In [5]:
# splitting our data into target and features

# defining features
X = final_df.drop(["song_status"], axis=1)

# defining target
y = final_df["song_status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# scaling the data

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# make our model

number_input_features = len(X_train.iloc[0])
layer1=50
layer2=15

model = tf.keras.models.Sequential()

# first layer

model.add(
    tf.keras.layers.Dense(units=layer1, input_dim=number_input_features, activation="relu")
)

# second layer

model.add(
    tf.keras.layers.Dense(units=layer2, activation="relu")
)

# output layer

model.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                750       
                                                                 
 dense_1 (Dense)             (None, 15)                765       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1531 (5.98 KB)
Trainable params: 1531 (5.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
fit_model = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
23/23 [==============================] - 1s 2ms/step - loss: 0.7008 - accuracy: 0.5448
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.5910
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6583 - accuracy: 0.6050
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6464 - accuracy: 0.6415
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6373 - accuracy: 0.6639
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6290 - accuracy: 0.6709
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6193 - accuracy: 0.6835
Epoch 8/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6115 - accuracy: 0.6947
Epoch 9/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6024 - accuracy: 0.7003
Epoch 10/100
23/23 [==============================] - 0s 2ms/step - loss: 0.5942 - accuracy: 0.7003
Epoch 11/

In [10]:
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.6350 - accuracy: 0.7113 - 175ms/epoch - 22ms/step
Loss: 0.635024905204773, Accuracy: 0.7112970948219299


In [11]:
prediction = model.predict(X_test_scaled)

8/8 [==============================] - 0s 2ms/step


In [12]:
prediction[40]

array([0.31812465], dtype=float32)